In [13]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [14]:
import copy
import datetime
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [47]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command, segment, block, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar) 

In [48]:
# celeri.plot_block_labels(segment, block, station)

In [51]:
def get_block_constraint_partials(block):
    """
    Partials for a priori block motion constraints.
    Essentially a set of eye(3) matrices
    """
    apriori_block_idx = np.where(block.apriori_flag.to_numpy() == 1)[0]
    print(apriori_block_idx)
    operator = np.zeros((3 * len(apriori_block_idx), 3 * len(block)))
    for i in range(len(apriori_block_idx)):
        start_row = 3 * i
        start_column = 3 * apriori_block_idx[i]
        operator[start_row:start_row+3, start_column:start_column+3] = np.eye(3)
    return operator


def block_constraints(assembly, block, command):
    """
    Applying a priori block motion constraints
    """
    block_constraint_partials = get_block_constraint_partials(block)
    assembly["index"]["block_constraints_idx"] = np.where(block.apriori_flag.to_numpy == 1)[0]
    assembly["data"]["n_block_constraints"] = 3 * len(assembly["index"]["block_constraints_idx"])
    print(block_constraint_partials.shape[0])
    assembly["data"]["block_constraints"] = np.zeros(block_constraint_partials.shape[0])
    assembly["sigma"]["block_constraints"] = np.zeros(block_constraint_partials.shape[0])
    if assembly["data"]["n_block_constraints"] > 0:
        (
            assembly["data"]["block_constraints"][0::3],
            assembly["data"]["block_constraints"][1::3],
            assembly["data"]["block_constraints"][2::3],
        ) = celeri.sph2cart(
            np.deg2rad(block.euler_lon[assembly["index"]["block_constraints_idx"]]),
            np.deg2rad(block.euler_lat[assembly["index"]["block_constraints_idx"]]),
            np.deg2rad(block.rotation_rate[assembly["index"]["block_constraints_idx"]]),
        )

    #    apbcov = stack3([deg_to_rad(Block.eulerLatSig(Index.blockCon)), deg_to_rad(Block.eulerLonSig(Index.blockCon)), deg_to_rad(Block.rotationRateSig(Index.blockCon))])
    #    apbsx, apbsy, apbsz = epoles_cov_to_omega_cov(apbx, apby, apbz, diag(apbcov))
    #    sigma.block_constraints[0::3] = apbsx
    #    sigma.block_constraints[1::3] = apbsy
    #    sigma.block_constraints[2::3] = apbsz

    assembly["sigma"]["sigma.block_constraint_weight"] = command["block_constraint_weight"]
    return assembly, block_constraint_partials 

assembly, block_constraint_partials = block_constraints(assembly, block, command)

[29]
3


In [32]:
n_blocks = 3
apriori_block_idx = [0, 2]
operator = np.zeros((3 * len(apriori_block_idx), 3 * n_blocks))
for i in range(len(apriori_block_idx)):
    start_row = 3 * i
    start_column = 3 * apriori_block_idx[i]
    operator[start_row:start_row+3, start_column:start_column+3] = np.eye(3)
print(operator)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [45]:
command.keys()

dict_keys(['file_name', 'reuse_elastic', 'reuse_elastic_file', 'save_kernels', 'poissons_ratio', 'unit_sigmas', 'locking_depth_flag2', 'locking_depth_flag3', 'locking_depth_flag4', 'locking_depth_flag5', 'locking_depth_override_flag', 'locking_depth_overide_value', 'apriori_block_name', 'patch_file_names', 'tri_smooth', 'pmag_tri_smooth', 'smooth_type', 'n_iterations', 'tri_edge', 'tri_depth_tolerance', 'tri_con_weight', 'strain_method', 'sar_file_name', 'sar_ramp', 'sar_weight', 'tri_slip_constraint_type', 'inversion_type', 'inversion_param01', 'inversion_param02', 'inversion_param03', 'inversion_param04', 'inversion_param05', 'dump_all', 'mogi_file_name', 'solution_method', 'ridge_param', 'tri_full_coupling', 'tvr_lambda', 'tri_slip_sign', 'n_eigs', 'segment_file_name', 'station_file_name', 'block_file_name', 'fault_resolution', 'station_data_weight', 'station_data_weight_min', 'station_data_weight_max', 'station_data_weight_steps', 'slip_con_weight', 'slip_con_weight_min', 'slip_con